# Improved Model


## Importing packages and data


In [23]:
from typing import Any

import pickle
import sklearn
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, TargetEncoder

In [24]:
df = pd.read_csv('../dataset/listings.csv')
df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2595,https://www.airbnb.com/rooms/2595,20241104040953,2024-11-04,city scrape,Skylit Midtown Castle Sanctuary,"Beautiful, spacious skylit studio in the heart...",Centrally located in the heart of Manhattan ju...,https://a0.muscache.com/pictures/miso/Hosting-...,2845,https://www.airbnb.com/users/show/2845,Jennifer,2008-09-09,"Woodstock, NY",A New Yorker since 2000! My passion is creatin...,within a day,90%,21%,f,https://a0.muscache.com/im/pictures/user/50fc5...,https://a0.muscache.com/im/pictures/user/50fc5...,Midtown,7.0,9.0,"['email', 'phone', 'work_email']",t,t,Neighborhood highlights,Midtown,Manhattan,40.75356,-73.98559,Entire rental unit,Entire home/apt,1,1.0,1 bath,0.0,1.0,"[""Fire extinguisher"", ""Smoke alarm"", ""Stove"", ...",$240.00,30,1125,30.0,30.0,1125.0,1125.0,30.0,1125.0,NaN,t,30,60,90,365,2024-11-04,49,0,0,2009-11-21,2022-06-21,4.68,4.73,4.63,4.77,4.80,4.81,4.40,NaN,f,3,3,0,0,0.27
1,6848,https://www.airbnb.com/rooms/6848,20241104040953,2024-11-04,city scrape,Only 2 stops to Manhattan studio,Comfortable studio apartment with super comfor...,NaN,https://a0.muscache.com/pictures/e4f031a7-f146...,15991,https://www.airbnb.com/users/show/15991,Allen & Irina,2009-05-06,"New York, NY",We love to travel. When we travel we like to s...,within a few hours,100%,100%,t,https://a0.muscache.com/im/users/15991/profile...,https://a0.muscache.com/im/users/15991/profile...,Williamsburg,1.0,1.0,"['email', 'phone']",t,t,NaN,Williamsburg,Brooklyn,40.70935,-73.95342,Entire rental unit,Entire home/apt,3,1.0,1 bath,2.0,1.0,"[""Fire extinguisher"", ""Smoke alarm"", ""Stove"", ...",$83.00,30,120,30.0,30.0,120.0,120.0,30.0,120.0,NaN,t,0,15,15,185,2024-11-04,195,4,1,2009-05-25,2024-10-05,4.58,4.59,4.85,4.85,4.80,4.69,4.58,NaN,f,1,1,0,0,1.04
2,6872,https://www.airbnb.com/rooms/6872,20241104040953,2024-11-04,city scrape,Uptown Sanctuary w/ Private Bath (Month to Month),This charming distancing-friendly month-to-mon...,This sweet Harlem sanctuary is a 10-20 minute ...,https://a0.muscache.com/pictures/miso/Hosting-...,16104,https://www.airbnb.com/users/show/16104,Kae,2009-05-07,"New York, NY",A former life in fashion and wellness has left...,a few days or more,30%,33%,f,https://a0.muscache.com/im/pictures/user/d865a...,https://a0.muscache.com/im/pictures/user/d865a...,East Harlem,2.0,2.0,"['email', 'phone', 'work_email']",t,t,Neighborhood highlights,East Harlem,Manhattan,40.80107,-73.94255,Private room in condo,Private room,1,1.0,1 shared bath,1.0,1.0,"[""Heating"", ""Washer"", ""Fire extinguisher"", ""Sm...",$65.00,30,180,30.0,30.0,180.0,180.0,30.0,180.0,NaN,t,23,53,83,83,2024-11-04,1,0,0,2022-06-05,2022-06-05,5.00,5.00,5.00,5.00,5.00,5.00,5.

### Configuring pandas


In [25]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

## Defining functions


In [26]:
def model_predict(model: BaseEstimator) -> float:
    prediction = model.predict(
        pd.DataFrame(
            {
                'name': ['Skylit Midtown Castle'],
                'host_id': 2845,
                'host_name': ['Jennifer'],
                'neighbourhood_group_cleansed': ['Manhatan'],
                'neighbourhood_cleansed': ['Midtown'],
                'latitude': [40.75362],
                'longitude': [-73.98377],
                'room_type': ['Entire home / apt'],
                'minimum_nights': [1],
                'number_of_reviews': [45],
                'last_review': ['2019-05-21'],
                'reviews_per_month': [0.38],
                'calculated_host_listings_count': [2],
                'availability_365': [355],
                'accommodates': None,
                'bathrooms_text': None,
                'review_scores_rating': None,
                'review_scores_location': None,
                'maximum_nights': None,
                'review_scores_checkin': None,
                'bedrooms': None,
                'property_type': None,
                'maximum_nights_avg_ntm': None,
                'review_scores_accuracy': None,
                'beds': None,
                'review_scores_value': None,
                'review_scores_communication': None,
                'review_scores_cleanliness': None,
                'minimum_nights_avg_ntm': None,
            },
        )
    )
    return float(prediction[0])

## Exploring Data


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37548 entries, 0 to 37547
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            37548 non-null  int64  
 1   listing_url                                   37548 non-null  object 
 2   scrape_id                                     37548 non-null  int64  
 3   last_scraped                                  37548 non-null  object 
 4   source                                        37548 non-null  object 
 5   name                                          37546 non-null  object 
 6   description                                   36504 non-null  object 
 7   neighborhood_overview                         20574 non-null  object 
 8   picture_url                                   37547 non-null  object 
 9   host_id                                       37548 non-null 

### Creating new DF only with essential features


In [28]:
processed_df = df[
    [
        'neighbourhood_cleansed',
        'neighbourhood_group_cleansed',
        'property_type',
        'room_type',
        'bathrooms_text',
        'price',
        'beds',
        'bedrooms',
        'accommodates',
        'minimum_nights',
        'maximum_nights',
        'minimum_nights_avg_ntm',
        'maximum_nights_avg_ntm',
        'availability_365',
        'number_of_reviews',
        'calculated_host_listings_count',
        'reviews_per_month',
        'review_scores_value',
        'review_scores_rating',
        'review_scores_checkin',
        'review_scores_location',
        'review_scores_accuracy',
        'review_scores_cleanliness',
        'review_scores_communication',
    ]
].copy()

In [29]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37548 entries, 0 to 37547
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   neighbourhood_cleansed          37548 non-null  object 
 1   neighbourhood_group_cleansed    37548 non-null  object 
 2   property_type                   37548 non-null  object 
 3   room_type                       37548 non-null  object 
 4   bathrooms_text                  37518 non-null  object 
 5   price                           22741 non-null  object 
 6   beds                            22596 non-null  float64
 7   bedrooms                        31565 non-null  float64
 8   accommodates                    37548 non-null  int64  
 9   minimum_nights                  37548 non-null  int64  
 10  maximum_nights                  37548 non-null  int64  
 11  minimum_nights_avg_ntm          37546 non-null  float64
 12  maximum_nights_avg_ntm          

In [30]:
processed_df.nunique()

neighbourhood_cleansed            223
neighbourhood_group_cleansed        5
property_type                      78
room_type                           4
bathrooms_text                     34
price                             978
beds                               18
bedrooms                           14
accommodates                       16
minimum_nights                    120
maximum_nights                    252
minimum_nights_avg_ntm            404
maximum_nights_avg_ntm            965
availability_365                  366
number_of_reviews                 486
calculated_host_listings_count     70
reviews_per_month                 804
review_scores_value               173
review_scores_rating              167
review_scores_checkin             138
review_scores_location            157
review_scores_accuracy            161
review_scores_cleanliness         186
review_scores_communication       150
dtype: int64

### Transforming Data


In [31]:
processed_df.isna().sum()

neighbourhood_cleansed                0
neighbourhood_group_cleansed          0
property_type                         0
room_type                             0
bathrooms_text                       30
price                             14807
beds                              14952
bedrooms                           5983
accommodates                          0
minimum_nights                        0
maximum_nights                        0
minimum_nights_avg_ntm                2
maximum_nights_avg_ntm                2
availability_365                      0
number_of_reviews                     0
calculated_host_listings_count        0
reviews_per_month                 11560
review_scores_value               11577
review_scores_rating              11560
review_scores_checkin             11575
review_scores_location            11578
review_scores_accuracy            11571
review_scores_cleanliness         11561
review_scores_communication       11566
dtype: int64

In [32]:
processed_df['price'] = (
    processed_df['price'].replace('[$,]', '', regex=True).astype(float)
)

In [33]:
processed_df.fillna({'price': processed_df['price'].mean()}, inplace=True)

## Modelling


### Creating pipelines


In [34]:
np.random.seed(42)

categorical_features = [
    'neighbourhood_cleansed',
    'neighbourhood_group_cleansed',
    'property_type',
    'room_type',
    'bathrooms_text',
]
numerical_features = [
    'beds',
    'bedrooms',
    'accommodates',
    'minimum_nights',
    'maximum_nights',
    'minimum_nights_avg_ntm',
    'maximum_nights_avg_ntm',
    'availability_365',
    'number_of_reviews',
    'calculated_host_listings_count',
    'reviews_per_month',
    'review_scores_value',
    'review_scores_rating',
    'review_scores_checkin',
    'review_scores_location',
    'review_scores_accuracy',
    'review_scores_cleanliness',
    'review_scores_communication',
]

categorical_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        (
            'target_encoding',
            TargetEncoder(categories='auto', target_type='continuous', smooth=0.2),
        ),
    ]
)

numerical_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaling', StandardScaler()),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ('categorical', categorical_transformer, categorical_features),
        ('numerical', numerical_transformer, numerical_features),
    ]
)

model = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('model', RandomForestRegressor()),
    ]
)

### Splitting data


In [35]:
X = processed_df.drop('price', axis=1)
y = processed_df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [36]:
param_grid = {
    'model__n_estimators': [100, 300, 500, 1000],
    'model__max_depth': [None, 10, 20, 30],
    'model__min_samples_split': [2, 5, 10, 15],
    'model__min_samples_leaf': [1, 2, 5, 10],
    'model__max_features': ['sqrt', 'log2', None],
    'model__bootstrap': [True, False],
}

In [37]:
rs_model = RandomizedSearchCV(model, param_grid, cv=5, verbose=2, n_jobs=5)
rs_model.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[CV] END model__bootstrap=True, model__max_depth=20, model__max_features=sqrt, model__min_samples_leaf=2, model__min_samples_split=15, model__n_estimators=100; total time=   2.0s
[CV] END model__bootstrap=True, model__max_depth=20, model__max_features=sqrt, model__min_samples_leaf=2, model__min_samples_split=15, model__n_estimators=100; total time=   2.0s
[CV] END model__bootstrap=True, model__max_depth=20, model__max_features=sqrt, model__min_samples_leaf=2, model__min_samples_split=15, model__n_estimators=100; total time=   2.0s
[CV] END model__bootstrap=True, model__max_depth=20, model__max_features=sqrt, model__min_samples_leaf=2, model__min_samples_split=15, model__n_estimators=100; total time=   2.0s
[CV] END model__bootstrap=True, model__max_depth=20, model__max_features=sqrt, model__min_samples_leaf=2, model__min_samples_split=15, model__n_estimators=100; total time=   2.0s
[CV] END model__bootstrap=True, model__max_depth=10, model__max_features=log2, model__min_samples_leaf=5,

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('categorical',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='most_frequent')),
                                                                                               ('target_encoding',
                                                                                                TargetEncoder(smooth=0.2,
                                                                                                              target_type='continuous'))]),
                                                                               ['neighbourhood_cleansed',
                                                                                'neighbourhood_group_cleansed',
                                                                                'property_type',
                                                                                'room_type',
                                                                                'bathrooms_...
                                                                                'review_scores_cleanliness',
                                                                                'review_scores_communication'])])),
                                             ('model',
                                              RandomForestRegressor())]),
                   n_jobs=5,
                   param_distributions={'model__bootstrap': [True, False],
                                        'model__max_depth': [None, 10, 20, 30],
                                        'model__max_features': ['sqrt', 'log2',
                                                                None],
                                        'model__min_samples_leaf': [1, 2, 5,
                                                                    10],
                                        'model__min_samples_split': [2, 5, 10,
                                                                     15],
                                        'model__n_estimators': [100, 300, 500,
                                                                1000]},
                   verbose=2)

In [38]:
score_test, score_train = rs_model.score(X_test, y_test), rs_model.score(
    X_train, y_train
)
score_test, score_train

(0.3854184573868533, 0.5681448073136126)

In [39]:
pickle.dump(rs_model, open('../models/model.pkl', 'wb'))

## Making predictions


In [41]:
model_predict(rs_model)

276.8228462563686